In [1]:
import numpy as np
from simulation.odorSimulation.aquaEnv import AquaEnv
from policy.QLearning import QLearning
from simulation.odorSimulation.currentAquaSimulation import *
from runner.loadLocalizeModel import getLocalizeModel


model_arg = {
    'file_name': "q_learning",
    # training parameter
    'er_size': 40,
    'n_ep_training': 0,
    'n_ep_evaluate': 1,

    #TODO change buffer ini to True
    'buffer_ini': True,

    #TODO change
    'print_while_learning': True,

    # #env parameter overwrite
    # 'episode_limit': 40,

    # RL parameter
    #TODO check decay
    'epsilon_decay': 150000,
    'bsize_update': 128,
    'gamma': 0.99,
    'grad_norm_clip': 10,
    'target_update_cycle': 5,

    # save parameter
    'save_interval': 2000,
    'save_model': True,
    'save_time': True,
    'evaluate_model':True,
}

# localize Model
localize_model = getLocalizeModel()
model_arg['belief_state_model'] = localize_model
env_arg['state_dim'] = localize_model.hid_dim
env_arg['episode_limit'] = 40


env = AquaEnv(env_arg, agent_arg, source_arg, wind_arg)
model = QLearning(env, model_arg, env_arg)
# model.showEpsilon()
model.learn()

30 40


100%|██████████| 1/1 [00:00<00:00,  1.02it/s]
0it [00:00, ?it/s]


In [2]:
batch = model.experience_replay.sample(40)

In [3]:
action, reward, done = batch['a'], batch['r'], batch['done']  # (bsize, )
state, state_next = batch['s'], batch['s_next']  # (bsize, obs_dim)

In [4]:
self = model
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_printoptions(precision=2, threshold=None, edgeitems=None, linewidth=None, profile=None, sci_mode=False)

In [5]:
bsize = 1
obs = self.env.reset()
action = torch.randint(self.action_dim, (bsize,))
state = self.getState(obs, action)

In [6]:
state

tensor([[[     0.59,     -0.00,     -1.00,     -0.01,      0.00,     -0.01,
               0.01,     -0.09,      0.00,      0.01,     -0.00,     -0.00,
              -1.00,     -0.00,     -1.00,     -0.11,      0.00,      0.76,
              -0.00,     -0.87,     -0.39,      0.24,      0.00,      0.00,
               0.00,      0.01,     -0.00,     -0.11,     -0.09,     -0.00,
              -0.00,      0.03,     -0.17,      0.00,      1.00,     -0.33,
               0.00,     -0.00,      0.00,     -0.00,     -0.00,      0.00,
              -0.89,     -0.62,     -0.00,     -1.00,     -0.18,      0.00,
              -0.40,      0.28,      1.00,     -0.00,     -0.34,      0.91,
               0.00,     -0.00,      0.07,      0.22,      1.00,      0.00,
              -1.00,     -0.00,      0.00,      0.00]]], device='cuda:0',
       grad_fn=<CudnnRnnBackward>)

In [8]:
action_prob = self.model(state.to(DEVICE))

prob, action = action_prob.max(dim=-1)

# # check e-greedy


In [10]:
action_prob

tensor([[[ 0.34, -0.10,  0.06, -0.33]]], device='cuda:0',
       grad_fn=<AddBackward0>)

In [12]:
use_epsilon = True
if use_epsilon:
    use_greedy = (torch.rand(bsize) < self.epsilon).to(torch.int).to(DEVICE)
    action_greedy = torch.randint(self.action_dim, (bsize,)).to(DEVICE)
    pick_action = torch.where(use_greedy == 1, action_greedy, action)
else:
    pick_action = action